In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🌾 Wereng Classification Model Training\n",
    "\n",
    "Notebook untuk training model klasifikasi hama wereng menggunakan Transfer Learning (MobileNetV2)\n",
    "\n",
    "**Dataset Structure:**\n",
    "```\n",
    "dataset/\n",
    "├── train/\n",
    "│   ├── wereng_coklat/\n",
    "│   ├── wereng_hijau/\n",
    "│   ├── wereng_punggung_putih/\n",
    "│   └── bukan_wereng/\n",
    "└── validation/\n",
    "    ├── wereng_coklat/\n",
    "    ├── wereng_hijau/\n",
    "    ├── wereng_punggung_putih/\n",
    "    └── bukan_wereng/\n",
    "```"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Import Libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import tensorflow as tf\n",
    "from tensorflow import keras\n",
    "from tensorflow.keras import layers, models\n",
    "from tensorflow.keras.applications import MobileNetV2\n",
    "from tensorflow.keras.preprocessing.image import ImageDataGenerator\n",
    "from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import os\n",
    "from datetime import datetime\n",
    "\n",
    "print(f\"TensorFlow version: {tf.__version__}\")\n",
    "print(f\"GPU available: {tf.config.list_physical_devices('GPU')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Configuration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Paths\n",
    "TRAIN_DIR = 'dataset/train'\n",
    "VAL_DIR = 'dataset/validation'\n",
    "MODEL_SAVE_PATH = 'app/models/wereng_classifier.h5'\n",
    "\n",
    "# Hyperparameters\n",
    "IMG_SIZE = (224, 224)\n",
    "BATCH_SIZE = 32\n",
    "EPOCHS = 50\n",
    "LEARNING_RATE = 0.0001\n",
    "\n",
    "# Classes\n",
    "CLASS_NAMES = [\n",
    "    'wereng_coklat',\n",
    "    'wereng_hijau', \n",
    "    'wereng_punggung_putih',\n",
    "    'bukan_wereng'\n",
    "]\n",
    "\n",
    "NUM_CLASSES = len(CLASS_NAMES)\n",
    "\n",
    "print(f\"Number of classes: {NUM_CLASSES}\")\n",
    "print(f\"Classes: {CLASS_NAMES}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Preparation & Augmentation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data augmentation untuk training\n",
    "train_datagen = ImageDataGenerator(\n",
    "    rescale=1./255,\n",
    "    rotation_range=20,\n",
    "    width_shift_range=0.2,\n",
    "    height_shift_range=0.2,\n",
    "    shear_range=0.2,\n",
    "    zoom_range=0.2,\n",
    "    horizontal_flip=True,\n",
    "    fill_mode='nearest'\n",
    ")\n",
    "\n",
    "# Hanya rescaling untuk validation\n",
    "val_datagen = ImageDataGenerator(rescale=1./255)\n",
    "\n",
    "# Load training data\n",
    "train_generator = train_datagen.flow_from_directory(\n",
    "    TRAIN_DIR,\n",
    "    target_size=IMG_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical',\n",
    "    classes=CLASS_NAMES,\n",
    "    shuffle=True\n",
    ")\n",
    "\n",
    "# Load validation data\n",
    "val_generator = val_datagen.flow_from_directory(\n",
    "    VAL_DIR,\n",
    "    target_size=IMG_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical',\n",
    "    classes=CLASS_NAMES,\n",
    "    shuffle=False\n",
    ")\n",
    "\n",
    "print(f\"\\nTraining samples: {train_generator.samples}\")\n",
    "print(f\"Validation samples: {val_generator.samples}\")\n",
    "print(f\"Class indices: {train_generator.class_indices}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualize Sample Images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualisasi beberapa gambar training\n",
    "def show_sample_images(generator, num_images=9):\n",
    "    images, labels = next(generator)\n",
    "    \n",
    "    plt.figure(figsize=(12, 12))\n",
    "    for i in range(min(num_images, len(images))):\n",
    "        plt.subplot(3, 3, i + 1)\n",
    "        plt.imshow(images[i])\n",
    "        class_idx = np.argmax(labels[i])\n",
    "        plt.title(CLASS_NAMES[class_idx])\n",
    "        plt.axis('off')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "show_sample_images(train_generator)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Build Model (Transfer Learning - MobileNetV2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_model(num_classes, img_size=(224, 224)):\n",
    "    \"\"\"\n",
    "    Create model using MobileNetV2 with transfer learning\n",
    "    \"\"\"\n",
    "    \n",
    "    # Load MobileNetV2 pre-trained on ImageNet\n",
    "    base_model = MobileNetV2(\n",
    "        input_shape=(*img_size, 3),\n",
    "        include_top=False,\n",
    "        weights='imagenet'\n",
    "    )\n",
    "    \n",
    "    # Freeze base model layers\n",
    "    base_model.trainable = False\n",
    "    \n",
    "    # Create new model\n",
    "    model = models.Sequential([\n",
    "        base_model,\n",
    "        layers.GlobalAveragePooling2D(),\n",
    "        layers.Dropout(0.5),\n",
    "        layers.Dense(256, activation='relu'),\n",
    "        layers.Dropout(0.3),\n",
    "        layers.Dense(num_classes, activation='softmax')\n",
    "    ])\n",
    "    \n",
    "    return model, base_model\n",
    "\n",
    "# Create model\n",
    "model, base_model = create_model(NUM_CLASSES, IMG_SIZE)\n",
    "\n",
    "# Compile model\n",
    "model.compile(\n",
    "    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),\n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]\n",
    ")\n",
    "\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Setup Callbacks"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create models directory if not exists\n",
    "os.makedirs('app/models', exist_ok=True)\n",
    "\n",
    "# Callbacks\n",
    "checkpoint = ModelCheckpoint(\n",
    "    MODEL_SAVE_PATH,\n",
    "    monitor='val_accuracy',\n",
    "    save_best_only=True,\n",
    "    mode='max',\n",
    "    verbose=1\n",
    ")\n",
    "\n",
    "early_stopping = EarlyStopping(\n",
    "    monitor='val_loss',\n",
    "    patience=10,\n",
    "    restore_best_weights=True,\n",
    "    verbose=1\n",
    ")\n",
    "\n",
    "reduce_lr = ReduceLROnPlateau(\n",
    "    monitor='val_loss',\n",
    "    factor=0.5,\n",
    "    patience=5,\n",
    "    min_lr=1e-7,\n",
    "    verbose=1\n",
    ")\n",
    "\n",
    "callbacks = [checkpoint, early_stopping, reduce_lr]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Train Model (Phase 1: Frozen Base)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"PHASE 1: Training with frozen base model\")\n",
    "print(\"=\"*50 + \"\\n\")\n",
    "\n",
    "history_phase1 = model.fit(\n",
    "    train_generator,\n",
    "    epochs=20,\n",
    "    validation_data=val_generator,\n",
    "    callbacks=callbacks,\n",
    "    verbose=1\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Fine-tuning (Phase 2: Unfreeze Some Layers)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"PHASE 2: Fine-tuning (unfreezing top layers)\")\n",
    "print(\"=\"*50 + \"\\n\")\n",
    "\n",
    "# Unfreeze the top layers of the base model\n",
    "base_model.trainable = True\n",
    "\n",
    "# Freeze all layers except the last 20\n",
    "for layer in base_model.layers[:-20]:\n",
    "    layer.trainable = False\n",
    "\n",
    "# Recompile with lower learning rate\n",
    "model.compile(\n",
    "    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),\n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]\n",
    ")\n",
    "\n",
    "print(f\"Number of trainable layers: {len([l for l in model.layers if l.trainable])}\")\n",
    "\n",
    "# Continue training\n",
    "history_phase2 = model.fit(\n",
    "    train_generator,\n",
    "    epochs=30,\n",
    "    initial_epoch=history_phase1.epoch[-1],\n",
    "    validation_data=val_generator,\n",
    "    callbacks=callbacks,\n",
    "    verbose=1\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Plot Training History"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_training_history(history1, history2=None):\n",
    "    \"\"\"\n",
    "    Plot training and validation accuracy/loss\n",
    "    \"\"\"\n",
    "    # Combine histories\n",
    "    if history2:\n",
    "        acc = history1.history['accuracy'] + history2.history['accuracy']\n",
    "        val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']\n",
    "        loss = history1.history['loss'] + history2.history['loss']\n",
    "        val_loss = history1.history['val_loss'] + history2.history['val_loss']\n",
    "    else:\n",
    "        acc = history1.history['accuracy']\n",
    "        val_acc = history1.history['val_accuracy']\n",
    "        loss = history1.history['loss']\n",
    "        val_loss = history1.history['val_loss']\n",
    "    \n",
    "    epochs_range = range(len(acc))\n",
    "    \n",
    "    plt.figure(figsize=(14, 5))\n",
    "    \n",
    "    # Plot accuracy\n",
    "    plt.subplot(1, 2, 1)\n",
    "    plt.plot(epochs_range, acc, label='Training Accuracy')\n",
    "    plt.plot(epochs_range, val_acc, label='Validation Accuracy')\n",
    "    plt.axvline(x=20, color='r', linestyle='--', label='Fine-tuning starts')\n",
    "    plt.legend(loc='lower right')\n",
    "    plt.title('Training and Validation Accuracy')\n",
    "    plt.xlabel('Epoch')\n",
    "    plt.ylabel('Accuracy')\n",
    "    \n",
    "    # Plot loss\n",
    "    plt.subplot(1, 2, 2)\n",
    "    plt.plot(epochs_range, loss, label='Training Loss')\n",
    "    plt.plot(epochs_range, val_loss, label='Validation Loss')\n",
    "    plt.axvline(x=20, color='r', linestyle='--', label='Fine-tuning starts')\n",
    "    plt.legend(loc='upper right')\n",
    "    plt.title('Training and Validation Loss')\n",
    "    plt.xlabel('Epoch')\n",
    "    plt.ylabel('Loss')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "plot_training_history(history_phase1, history_phase2)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Evaluate Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate on validation set\n",
    "print(\"\\nEvaluating model on validation set...\")\n",
    "val_loss, val_accuracy, val_top2_acc = model.evaluate(val_generator, verbose=1)\n",
    "\n",
    "print(f\"\\n{'='*50}\")\n",
    "print(f\"Validation Loss: {val_loss:.4f}\")\n",
    "print(f\"Validation Accuracy: {val_accuracy*100:.2f}%\")\n",
    "print(f\"Validation Top-2 Accuracy: {val_top2_acc*100:.2f}%\")\n",
    "print(f\"{'='*50}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 11. Confusion Matrix & Classification Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "import seaborn as sns\n",
    "\n",
    "# Get predictions\n",
    "val_generator.reset()\n",
    "predictions = model.predict(val_generator, verbose=1)\n",
    "y_pred = np.argmax(predictions, axis=1)\n",
    "y_true = val_generator.classes\n",
    "\n",
    "# Confusion matrix\n",
    "cm = confusion_matrix(y_true, y_pred)\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', \n",
    "            xticklabels=CLASS_NAMES, \n",
    "            yticklabels=CLASS_NAMES)\n",
    "plt.title('Confusion Matrix')\n",
    "plt.ylabel('True Label')\n",
    "plt.xlabel('Predicted Label')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Classification report\n",
    "print(\"\\nClassification Report:\")\n",
    "print(\"=\"*60)\n",
    "report = classification_report(y_true, y_pred, target_names=CLASS_NAMES)\n",
    "print(report)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 12. Test Predictions on Sample Images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def predict_and_visualize(model, generator, num_samples=6):\n",
    "    \"\"\"\n",
    "    Visualize predictions on sample images\n",
    "    \"\"\"\n",
    "    images, labels = next(generator)\n",
    "    predictions = model.predict(images[:num_samples])\n",
    "    \n",
    "    plt.figure(figsize=(15, 10))\n",
    "    for i in range(num_samples):\n",
    "        plt.subplot(2, 3, i